# Compress PA and SA models alone

In [1]:
import os 
opath = os.getcwd()

In [2]:
import cobra

# *P. aeruginosa*

In [3]:
pa = cobra.io.read_sbml_model('iPae_1146_medium.xml')

In [4]:
model = pa.copy()

In [5]:
import efmtool_link.efmtool_intern as efmtool_intern
import numpy
import cobra
import efmtool_link.efmtool_extern as efmtool_extern
import efmtool_link.efmtool4cobra as efmtool4cobra

openjdk version "11.0.19" 2023-04-18
OpenJDK Runtime Environment (build 11.0.19+7-post-Ubuntu-0ubuntu118.04.1)
OpenJDK 64-Bit Server VM (build 11.0.19+7-post-Ubuntu-0ubuntu118.04.1, mixed mode, sharing)


In [6]:
from pathlib import Path
def flux_variability_analysis(model: cobra.Model, loopless=False, fraction_of_optimum=0.0,
                              processes=None, results_cache_dir: Path=None, fva_hash=None, print_func=print):
    # all bounds in the model must be finite because the COBRApy FVA treats unbounded results as errors
    if results_cache_dir is not None:
        fva_hash.update(pickle.dumps((loopless, fraction_of_optimum, model.tolerance))) # integrate solver tolerances?
        if fraction_of_optimum > 0:
            fva_hash.update(pickle.dumps(model.reactions.list_attr("objective_coefficient")))
            fva_hash.update(model.objective_direction.encode())
        file_path = results_cache_dir / (model.id+"_FVA_"+fva_hash.hexdigest())
        fva_result = None
        if Path.exists(file_path):
            try:
                fva_result = pandas.read_pickle(file_path)
                print_func("Loaded FVA result from", str(file_path))
            except:
                print_func("Loading FVA result from", str(file_path), "failed, running FVA.")
        else:
            print_func("No cached result available, running FVA...")
        if fva_result is None:
            fva_result = cobra.flux_analysis.flux_variability_analysis(model, reaction_list=None, loopless=loopless,
                                                             fraction_of_optimum=fraction_of_optimum,
                                                             pfba_factor=None, processes=processes)
            try:
                fva_result.to_pickle(file_path)
                print_func("Saved FVA result to ", str(file_path))
            except:
                print_func("Failed to write FVA result to ", str(file_path))
        return fva_result
    else:
        return cobra.flux_analysis.flux_variability_analysis(model, reaction_list=None, loopless=loopless,
                                                             fraction_of_optimum=fraction_of_optimum,
                                                             pfba_factor=None, processes=processes)

In [7]:
def fva_model(compressed_model):
    fva_tolerance=1e-9
    with model as fva: # can be skipped when a compressed model is available
        # when include_model_bounds=False modify bounds so that only reversibilites are used?
        # fva.solver = 'glpk_exact' # too slow for large models
        fva.tolerance = fva_tolerance
        fva.objective = model.problem.Objective(0.0)
        if fva.problem.__name__ == 'optlang.glpk_interface':
            # should emulate setting an optimality tolerance (which GLPK simplex does not have)
            fva.solver.configuration._smcp.meth = GLP_DUAL
            fva.solver.configuration._smcp.tol_dj = fva_tolerance
        elif fva.problem.__name__ == 'optlang.coinor_cbc_interface':
            fva.solver.problem.opt_tol = fva_tolerance
        fva_res = flux_variability_analysis(fva, fraction_of_optimum=0.0, processes=1, 
            results_cache_dir=None, fva_hash=None)
    for i in range(fva_res.values.shape[0]): # assumes the FVA results are oreduced_matrixered same as the model reactions
        if abs(fva_res.values[i, 0]) > fva_tolerance: # resolve with glpk_exact?
            compressed_model.reactions[i].lower_bound = fva_res.values[i, 0]
        else:
            compressed_model.reactions[i].lower_bound = 0
        if abs(fva_res.values[i, 1]) > fva_tolerance: # resolve with glpk_exact?
            compressed_model.reactions[i].upper_bound = fva_res.values[i, 1]
        else:
            compressed_model.reactions[i].upper_bound = 0
    return compressed_model

In [8]:
# network compression (currently combination of reaction subsets only)
# IMPORTANT: the model is modified by this function, if you want to keep the full model copy it first
model = fva_model(model)
subT = efmtool4cobra.compress_model_sympy(model) # subT is a matrix for conversion of flux vectors between the full and compressed model
reduced_matrix = cobra.util.array.create_stoichiometric_matrix(model, array_type='lil')
# model compression makes sure that irreversible reactions always point in the forwareduced_matrix direction
rev_reduced_matrix = [int(r.reversibility) for r in model.reactions]
len(model.reactions)

Flipped SULR
Flipped HDER4
Flipped HBUR1
Flipped HHYR2
Flipped HOCR3
Flipped HHDR7
Flipped HDDR5
Flipped HTDR6
Flipped SHK3D
Flipped DHFR
Flipped P5CR
Flipped BDH
Flipped UAPGR
Flipped ILEDHr
Flipped DXPRI
Flipped HSDy
Flipped DHDPRy
Flipped KAT5
Flipped KAT3
Flipped KAT6
Flipped KAT7
Flipped KAT4
Flipped HXANt2
Flipped G1SAT
Flipped PACPT
Flipped ECOAH5
Flipped ECOAH7
Flipped ECOAH6
Flipped ECOAH4
Flipped ECOAH2
Flipped ECOAH3
Flipped ABUTt2
Flipped KAT2
Flipped XANt2
Flipped ALLNt2
Flipped ADNt2
Flipped CYTDt2
Flipped DADNt2
Flipped DCYTt2
Flipped INSt2
Flipped THMDt2
Flipped URIt2
Flipped ITACt
Flipped IPPMIa
Flipped FRUpts
Flipped ACGApts
Flipped HIS5
Flipped ACLS
Flipped FE3abc
Flipped CHLt2
Flipped APATr
Flipped 3OXACOA
Flipped PCAB
Flipped RPI
Flipped FBA
Flipped ECOAH9
Flipped ACONCt
Flipped CITMCOAH
Flipped CITMCOAL
Flipped UREA2t2
Flipped 2MBDT
Flipped 3MBT
Flipped ACOACAT
Flipped ECOAH1
Flipped NO3t2
Flipped PTRCORNt7
Flipped OCOAT1
Flipped ACACt2
Flipped BNBt
Flipped MGCH
F

/home/maxime/anaconda3/envs/cnapy/lib/python3.10/site-packages/cobra/core/group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")


470

In [9]:
reduced_matrix

<252x470 sparse matrix of type '<class 'numpy.float64'>'
	with 2402 stored elements in List of Lists format>

In [10]:
print(sum(rev_reduced_matrix), 'reversible reactions and', len(rev_reduced_matrix) - sum(rev_reduced_matrix), 'irreversible')

156 reversible reactions and 314 irreversible


In [11]:
original_model = pa.copy()
constrained_model = pa.copy()

In [12]:
import pandas
names = lambda l: list(map(lambda x: x.id, l))

In [13]:
df = pandas.DataFrame.sparse.from_spmatrix(reduced_matrix, index=names(model.metabolites), columns=names(model.reactions))
df

,ACOAD4f,ACOAD7f,ACOAD3f,ACOAD1f,HACD1,IDP1_1,G3PD,MMSAD3,THD2,ABUTD,...,EX_ptrc_e,EX_pyr_e,EX_succ_e,EX_urea_e,PPPGOan,EX_GLCN_e,EX_2KGLCN_e,EX_4abut_e,ATPM,DSFSc
fad_c,-3.0,-1.0,-2.0,-1.0,0.0,0.0,-1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fadh2_c,3.0,1.0,2.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
pmtcoa_c,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
tdcoa_c,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
btcoa_c,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
acglu_c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26dap_M_c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ppp9_c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
mg2_e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
import io
with io.StringIO() as output:
    for column in df.columns:
        dfc = df[column]
        ftime=True
        print(column, ': ', end='', file=output, flush=True)
        for k, v in dfc[dfc < 0].items():
            if ftime:
                ftime=False
            else:
                print('+', end=' ', file=output, flush=True)
            print(-v, k, end=' ', file=output, flush=True)
        print('=' if model.reactions.get_by_id(column).reversibility else '=>', end=' ', file=output, flush=True)
        ftime=True
        for k, v in dfc[dfc > 0].items():
            if ftime:
                ftime=False
            else:
                print('+', end=' ', file=output, flush=True)
            print(v, k, end=' ', file=output, flush=True)
        print(file=output, flush=True)
    lines_r = output.getvalue()
print(lines_r)

ACOAD4f : 3.0 fad_c + 1.0 tdcoa_c + 2.0 nad_c + 3.0 h2o_c + 2.0 coa_c => 3.0 fadh2_c + 2.0 nadh_c + 2.0 h_c + 1.0 3hdcoa_c + 2.0 accoa_c 
ACOAD7f : 1.0 fad_c + 1.0 pmtcoa_c + 1.0 nad_c + 1.0 h2o_c + 1.0 coa_c => 1.0 fadh2_c + 1.0 tdcoa_c + 1.0 nadh_c + 1.0 h_c + 1.0 accoa_c 
ACOAD3f : 2.0 fad_c + 3.0 nad_c + 1.0 3hdcoa_c + 2.0 h2o_c + 3.0 coa_c => 2.0 fadh2_c + 1.0 btcoa_c + 3.0 nadh_c + 3.0 h_c + 3.0 accoa_c 
ACOAD1f : 1.0 fad_c + 1.0 btcoa_c => 1.0 fadh2_c + 1.0 b2coa_c 
HACD1 : 1.0 nad_c + 1.0 3hbcoa_c = 1.0 nadh_c + 1.0 h_c + 1.0 aacoa_c 
IDP1_1 : 1.0 nadp_c + 1.0 icit_c = 1.0 nadph_c + 1.0 co2_c + 1.0 akg_c 
G3PD : 1.0 fad_c + 1.0 gly3p_c = 1.0 fadh2_c + 1.0 dhap_c 
MMSAD3 : 1.0 nad_c + 1.0 h_c + 1.0 coa_c + 1.0 asp__L_c + 1.0 pyr_c => 1.0 nadh_c + 2.0 co2_c + 1.0 accoa_c + 1.0 ala__L_c 
THD2 : 1.0 nadh_c + 1.0 nadp_c + 2.0 h_e = 1.0 nad_c + 2.0 h_c + 1.0 nadph_c 
ABUTD : 1.0 nad_c + 1.0 h2o_c + 1.0 4abutn_c => 1.0 nadh_c + 2.0 h_c + 1.0 4abut_c 
HXAND : 1.0 nad_c + 1.0 h2o_c + 1.

In [15]:
rxn_in_sub = [numpy.where(subT[:, i])[0] for i in range(subT.shape[1])]

In [16]:
for i, reactions in enumerate(model.reactions):
    print(f'rsub_{i+1}', list(map(lambda x: constrained_model.reactions[x].id, rxn_in_sub[i])))

rsub_1 ['ACOAD4f', 'ACOAD5f', 'ACOAD6f', 'HACD5', 'HACD6', 'KAT5', 'KAT6', 'ECOAH5', 'ECOAH6', 'ECOAH4']
rsub_2 ['ACOAD7f', 'HACD7', 'KAT7', 'ECOAH7']
rsub_3 ['ACOAD3f', 'ACOAD2f', 'HACD4', 'HACD2', 'HACD3', 'KAT3', 'KAT4', 'ECOAH2', 'ECOAH3', 'KAT2']
rsub_4 ['ACOAD1f']
rsub_5 ['HACD1']
rsub_6 ['IDP1_1', 'IDP1_2']
rsub_7 ['G3PD']
rsub_8 ['MMSAD3', 'APATr', 'ASP1DC']
rsub_9 ['THD2']
rsub_10 ['ABUTD']
rsub_11 ['HXAND']
rsub_12 ['XAND', 'URIC']
rsub_13 ['FRD']
rsub_14 ['SUCD4']
rsub_15 ['SDHU9']
rsub_16 ['GCCc', 'GCCb', 'GCCa']
rsub_17 ['SULR', 'SADT2', 'APSR', 'EX_so4_e']
rsub_18 ['ALD2', 'GLY1']
rsub_19 ['GAPDy1']
rsub_20 ['NADH11']
rsub_21 ['NADHU9H5']
rsub_22 ['NADH6']
rsub_23 ['ZWF1']
rsub_24 ['G6PDH']
rsub_25 ['GND']
rsub_26 ['MDH']
rsub_27 ['EAR40y']
rsub_28 ['EAR100y']
rsub_29 ['EAR120y']
rsub_30 ['EAR160y']
rsub_31 ['EAR60y']
rsub_32 ['EAR80y']
rsub_33 ['EAR140y']
rsub_34 ['HDER4']
rsub_35 ['GPD1']
rsub_36 ['G3PD2']
rsub_37 ['EAR40x']
rsub_38 ['EAR100x']
rsub_39 ['EAR120x']
rsub_

In [17]:
dicto_rsub = {}
biomass = None
for i, reactions in enumerate(model.reactions):
    lm = list(map(lambda x: constrained_model.reactions[x].id, rxn_in_sub[i]))
    if 'PAO1_Biomass' in lm:
        biomass = f'rsub_{i+1}'
    for r in lm:
        dicto_rsub[r] = f'rsub_{i+1}'
print(biomass)

rsub_51


In [18]:
lines = ''
nb_ext = 1
for i, column in enumerate(df.columns):
    dfc = df[column]
    ftime=True
    lines += f'rsub_{i+1}' + ' : '
    for k, v in dfc[dfc < 0].items():
        if ftime:
            ftime=False
        else:
            lines += ' + '
        lines += str(-v) + ' ' + k
    if ftime:
        lines += f'ext_{nb_ext}'
        nb_ext += 1
    lines += ' = ' if model.reactions.get_by_id(column).reversibility else ' => '
    ftime=True
    for k, v in dfc[dfc > 0].items():
        if ftime:
            ftime=False
        else:
            lines += ' + '
        lines += str(v) + ' ' + k
    if ftime:
        lines += f'ext_{nb_ext}'
        nb_ext += 1
    lines += '\n'
lines = '-METEXT\n' + ' '.join(['ext_' + str(i+1) for i in range(nb_ext-1)]) + '\n' + '\n-CAT\n' + lines
print(lines)

-METEXT
ext_1 ext_2 ext_3 ext_4 ext_5 ext_6 ext_7 ext_8 ext_9 ext_10 ext_11 ext_12 ext_13 ext_14 ext_15 ext_16 ext_17 ext_18 ext_19 ext_20 ext_21 ext_22 ext_23 ext_24 ext_25 ext_26 ext_27 ext_28 ext_29 ext_30 ext_31 ext_32 ext_33 ext_34 ext_35 ext_36

-CAT
rsub_1 : 3.0 fad_c + 1.0 tdcoa_c + 2.0 nad_c + 3.0 h2o_c + 2.0 coa_c => 3.0 fadh2_c + 2.0 nadh_c + 2.0 h_c + 1.0 3hdcoa_c + 2.0 accoa_c
rsub_2 : 1.0 fad_c + 1.0 pmtcoa_c + 1.0 nad_c + 1.0 h2o_c + 1.0 coa_c => 1.0 fadh2_c + 1.0 tdcoa_c + 1.0 nadh_c + 1.0 h_c + 1.0 accoa_c
rsub_3 : 2.0 fad_c + 3.0 nad_c + 1.0 3hdcoa_c + 2.0 h2o_c + 3.0 coa_c => 2.0 fadh2_c + 1.0 btcoa_c + 3.0 nadh_c + 3.0 h_c + 3.0 accoa_c
rsub_4 : 1.0 fad_c + 1.0 btcoa_c => 1.0 fadh2_c + 1.0 b2coa_c
rsub_5 : 1.0 nad_c + 1.0 3hbcoa_c = 1.0 nadh_c + 1.0 h_c + 1.0 aacoa_c
rsub_6 : 1.0 nadp_c + 1.0 icit_c = 1.0 nadph_c + 1.0 co2_c + 1.0 akg_c
rsub_7 : 1.0 fad_c + 1.0 gly3p_c = 1.0 fadh2_c + 1.0 dhap_c
rsub_8 : 1.0 nad_c + 1.0 h_c + 1.0 coa_c + 1.0 asp__L_c + 1.0 pyr_c => 

In [19]:
path = f'{opath}/sa_pa/'

In [20]:
txtfile = path + 'pa_reduced.txt'

In [21]:
with open(txtfile, 'w') as f:
    f.writelines(lines)

In [22]:
ri = lambda i: original_model.reactions[i].id

In [23]:
subname = lambda i: 'rsub_{}'.format(i+1)
subnamerev = lambda i: 'rsub_{}_rev'.format(i+1)
aq = lambda x: '"{}"'.format(x)
aqrev = lambda x: '"{}_rev"'.format(x)
rev = lambda x: '{}_rev'.format(x)  
lines = 'subset(' + ';'.join([aq(subname(i)) for i in range(len(df.columns))]) + ').\n'
lines += 'subset(' + ';'.join([aq(subnamerev(i)) for i in range(len(df.columns))]) + ').\n'
dicto = {}

for r, ls in enumerate(rxn_in_sub):
    dicto[subname(r)] = {'reacs': [ri(rk) if subT[rk, r] >= 0 else rev(ri(rk)) for rk in ls],
         'coeffs': [abs(subT[rk, r]) for rk in ls],
         'full': [(ri(rk), subT[rk, r]) for rk in ls]}
    if model.reactions.get_by_any(r)[0].reversibility:
        dicto[subnamerev(r)] = {'reacs': [ri(rk) if subT[rk, r] < 0 else rev(ri(rk)) for rk in ls],
             'coeffs': [abs(subT[rk, r]) for rk in ls],
             'full': [(ri(rk), -subT[rk, r]) for rk in ls]}
    for rk in ls:
        fk = subT[rk, r]
        lines += 'coefficient(' + aq(subname(r)) + ',' + ((aq(ri(rk))) if fk >= 0 else aqrev(ri(rk))) + ',' + aq(abs(fk)) + ').\n'
        lines += 'coefficient(' + aq(subnamerev(r)) + ',' + ((aqrev(ri(rk))) if fk >= 0 else aq(ri(rk))) + ',' + aq(abs(fk)) + ').\n'

In [24]:
lp4file = path + 'pa_reactionSubsets.lp4'
subfile = path + 'pa_reactionSubsets.txt'

In [25]:
with open(lp4file, 'w') as f:
    f.writelines(lines)

In [26]:
with open(subfile, 'w') as f:
    f.write(str(dicto))

In [27]:
mcssubname = lambda i: '"mcs_rsub_{}"'.format(i+1)
mcssubnamerev = lambda i: '"mcs_rsub_{}_rev"'.format(i+1)
aq = lambda x: '"{}"'.format(x)
aqrev = lambda x: '"{}_rev"'.format(x)    
lines = 'subset(' + ';'.join([mcssubname(i) for i in range(len(df.columns))]) + ').\n'
lines += 'subset(' + ';'.join([mcssubnamerev(i) for i in range(len(df.columns))]) + ').\n'

for r, ls in enumerate(rxn_in_sub):
    for rk in ls:
        fk = subT[rk, r]
        lines += 'coefficient(' + mcssubname(r) + ',' + ((aq(ri(rk))) if fk >= 0 else aqrev(ri(rk))) + ',' + aq(abs(fk)) + ').\n'
        lines += 'coefficient(' + mcssubnamerev(r) + ',' + ((aqrev(ri(rk))) if fk >= 0 else aq(ri(rk))) + ',' + aq(abs(fk)) + ').\n'

In [28]:
lp4file = path + 'pa_mcsReactionSubsets.lp4'

In [29]:
with open(lp4file, 'w') as f:
    f.writelines(lines)

In [30]:
len(original_model.reactions)

1495

In [31]:
lines = '&dom{LB..UB} = flux(R) :- reaction(R); bounds(R, LB, UB).\n'
for i, r in enumerate(model.reactions):
    if r.upper_bound < 0:
        lines += 'bounds(' + aq(subnamerev(i)) + ',' + aq(-r.lower_bound) + ',' + aq(-r.upper_bound) + ').\n'
    elif r.lower_bound < 0:
        lines += 'bounds(' + aq(subname(i)) + ',' + aq(0) + ',' + aq(r.upper_bound) + ').\n'
        lines += 'bounds(' + aq(subnamerev(i)) + ',' + aq(0) + ',' + aq(-r.lower_bound) + ').\n'
    else:
        lines += 'bounds(' + aq(subname(i)) + ',' + aq(r.lower_bound) + ',' + aq(r.upper_bound) + ').\n'

In [32]:
bndfile = path + 'pa_reduced_bounds.lp4'

In [33]:
with open(bndfile, 'w') as f:
    f.writelines(lines)

In [34]:
from os import chdir
chdir(f'{opath}/../mparser/')

In [35]:
import mparser

In [36]:
txtfile=f'{path}pa_reduced.txt'
pklfile=f'{path}pa_reduced.pkl'
aspfile=f'{path}pa_reduced.lp4'
sbmlfile=f'{path}pa_reduced.xml'
mcs_pklfile=f'{path}pa_reduced_mcs.pkl'
mcs_aspfile=f'{path}pa_reduced_mcs.lp4'
mcs_sbmlfile=f'{path}pa_reduced_mcs.xml'

In [37]:
mparser.convert(mparser.conversion.InputFormatType.TXT, txtfile , mparser.conversion.OutputFormatType.PKL, pklfile, False, [], False)
mparser.convert(mparser.conversion.InputFormatType.TXT, txtfile , mparser.conversion.OutputFormatType.ASP, aspfile, False, [], False)
mparser.convert(mparser.conversion.InputFormatType.TXT, txtfile , mparser.conversion.OutputFormatType.SBML, sbmlfile, False, [], False)

252 internal metabolites
36 external metabolites
470 total reactions
36 exchange reactions
156 reversible reactions
252 internal metabolites
36 external metabolites
470 total reactions
36 exchange reactions
156 reversible reactions
252 internal metabolites
36 external metabolites
470 total reactions
36 exchange reactions
156 reversible reactions


In [38]:
mparser.convert(mparser.conversion.InputFormatType.TXT, txtfile , mparser.conversion.OutputFormatType.PKL, mcs_pklfile, to_dual_mcs=True, target_reactions=[biomass], ballerstein=False)
mparser.convert(mparser.conversion.InputFormatType.TXT, txtfile , mparser.conversion.OutputFormatType.ASP, mcs_aspfile, to_dual_mcs=True, target_reactions=[biomass], ballerstein=False)
mparser.convert(mparser.conversion.InputFormatType.TXT, txtfile , mparser.conversion.OutputFormatType.SBML, mcs_sbmlfile, to_dual_mcs=True, target_reactions=[biomass], ballerstein=False)

252 internal metabolites
36 external metabolites
470 total reactions
36 exchange reactions
156 reversible reactions


/home/maxime/Bureau/Thesis/wksp/github/paSAmcs/mparser/mparser/meta_network.py:202: UserWarning: No external metabolite definition
  self.structures_check()
/home/maxime/Bureau/Thesis/wksp/github/paSAmcs/mparser/mparser/meta_network.py:43: UserWarning: Found reactions with missing overall metabolite production or consumption
  warnings.warn('Found reactions with missing overall metabolite production or consumption')


-- Dual network --
470 internal metabolites
0 external metabolites
723 total reactions
36 exchange reactions
408 reversible reactions
252 internal metabolites
36 external metabolites
470 total reactions
36 exchange reactions
156 reversible reactions
-- Dual network --
470 internal metabolites
0 external metabolites
723 total reactions
36 exchange reactions
408 reversible reactions
252 internal metabolites
36 external metabolites
470 total reactions
36 exchange reactions
156 reversible reactions
-- Dual network --
470 internal metabolites
0 external metabolites
723 total reactions
36 exchange reactions
408 reversible reactions


In [39]:
efmcfile=f'{path}pa_reduced.efmc'

In [40]:
from prepare_efm_checker import compute_efm_checker

In [41]:
compute_efm_checker(pklfile, efmcfile, None, True)

/home/maxime/Bureau/Thesis/wksp/github/paSAmcs/mparser/aspsmatrix.py:219: UserWarning: large model; skipping nullspace computation
  warnings.warn("large model; skipping nullspace computation")


In [42]:
chdir(f'{opath}')

# *S. aureus*

In [43]:
chdir(f'{path}..')

In [44]:
sa = cobra.io.read_sbml_model('iYS854_medium.xml')

In [45]:
model = sa.copy()

In [46]:
import efmtool_link.efmtool_intern as efmtool_intern
import numpy
import cobra
import efmtool_link.efmtool_extern as efmtool_extern
import efmtool_link.efmtool4cobra as efmtool4cobra

In [47]:
from pathlib import Path
from os import chdir
chdir('../mparser')

In [48]:
from pathlib import Path
def flux_variability_analysis(model: cobra.Model, loopless=False, fraction_of_optimum=0.0,
                              processes=None, results_cache_dir: Path=None, fva_hash=None, print_func=print):
    # all bounds in the model must be finite because the COBRApy FVA treats unbounded results as errors
    if results_cache_dir is not None:
        fva_hash.update(pickle.dumps((loopless, fraction_of_optimum, model.tolerance))) # integrate solver tolerances?
        if fraction_of_optimum > 0:
            fva_hash.update(pickle.dumps(model.reactions.list_attr("objective_coefficient")))
            fva_hash.update(model.objective_direction.encode())
        file_path = results_cache_dir / (model.id+"_FVA_"+fva_hash.hexdigest())
        fva_result = None
        if Path.exists(file_path):
            try:
                fva_result = pandas.read_pickle(file_path)
                print_func("Loaded FVA result from", str(file_path))
            except:
                print_func("Loading FVA result from", str(file_path), "failed, running FVA.")
        else:
            print_func("No cached result available, running FVA...")
        if fva_result is None:
            fva_result = cobra.flux_analysis.flux_variability_analysis(model, reaction_list=None, loopless=loopless,
                                                             fraction_of_optimum=fraction_of_optimum,
                                                             pfba_factor=None, processes=processes)
            try:
                fva_result.to_pickle(file_path)
                print_func("Saved FVA result to ", str(file_path))
            except:
                print_func("Failed to write FVA result to ", str(file_path))
        return fva_result
    else:
        return cobra.flux_analysis.flux_variability_analysis(model, reaction_list=None, loopless=loopless,
                                                             fraction_of_optimum=fraction_of_optimum,
                                                             pfba_factor=None, processes=processes)

In [49]:
def fva_model(compressed_model):
    fva_tolerance=1e-9
    with model as fva: # can be skipped when a compressed model is available
        # when include_model_bounds=False modify bounds so that only reversibilites are used?
        # fva.solver = 'glpk_exact' # too slow for large models
        fva.tolerance = fva_tolerance
        fva.objective = model.problem.Objective(0.0)
        if fva.problem.__name__ == 'optlang.glpk_interface':
            # should emulate setting an optimality tolerance (which GLPK simplex does not have)
            fva.solver.configuration._smcp.meth = GLP_DUAL
            fva.solver.configuration._smcp.tol_dj = fva_tolerance
        elif fva.problem.__name__ == 'optlang.coinor_cbc_interface':
            fva.solver.problem.opt_tol = fva_tolerance
        fva_res = flux_variability_analysis(fva, fraction_of_optimum=0.0, processes=1, 
            results_cache_dir=None, fva_hash=None)
    for i in range(fva_res.values.shape[0]): # assumes the FVA results are oreduced_matrixered same as the model reactions
        if abs(fva_res.values[i, 0]) > fva_tolerance: # resolve with glpk_exact?
            compressed_model.reactions[i].lower_bound = fva_res.values[i, 0]
        else:
            compressed_model.reactions[i].lower_bound = 0
        if abs(fva_res.values[i, 1]) > fva_tolerance: # resolve with glpk_exact?
            compressed_model.reactions[i].upper_bound = fva_res.values[i, 1]
        else:
            compressed_model.reactions[i].upper_bound = 0
    return compressed_model

In [50]:
# network compression (currently combination of reaction subsets only)
# IMPORTANT: the model is modified by this function, if you want to keep the full model copy it first
model = fva_model(model)
subT = efmtool4cobra.compress_model_sympy(model) # subT is a matrix for conversion of flux vectors between the full and compressed model
reduced_matrix = cobra.util.array.create_stoichiometric_matrix(model, array_type='lil')
# model compression makes sure that irreversible reactions always point in the forwareduced_matrix direction
rev_reduced_matrix = [int(r.reversibility) for r in model.reactions]
len(model.reactions)

Flipped ACALD
Flipped ACALDt
Flipped ACGApts
Flipped 3GMPtex_1
Flipped ACLS_b
Flipped 3HOXTPP
Flipped AGPR
Flipped 3NUCLE4
Flipped 3OAR120_2
Flipped 3OAR160_2
Flipped 3OAR80_2
Flipped AIRCr
Flipped AIRC3
Flipped ACOTA
Flipped ALAR
Flipped ALATA_D
Flipped ACtr
Flipped ALCD19
Flipped BTDD_RR
Flipped BTNTe
Flipped BTS3r
Flipped BGLK_1
Flipped CYStec
Flipped CYTK2
Flipped DADNt2
Flipped DCYTt2
Flipped DINSt2
Flipped DGSNt2
Flipped DHORTS
Flipped EX_gly_e
Flipped EX_ala__L_e
Flipped EX_gua_e
Flipped EX_asn__L_e
Flipped EX_k_e
Flipped EX_btn_e
Flipped EX_ca2_e
Flipped EX_cl_e
Flipped EX_csn_e
Flipped EX_cys__L_e
Flipped EX_mg2_e
Flipped EX_mn2_e
Flipped EX_na1_e
Flipped EX_fe2_e
Flipped EX_nac_e
Flipped EX_o2_e
Flipped EX_pnto__R_e
Flipped EX_pro__L_e
Flipped EX_pydx_e
Flipped EX_glc__D_e
Flipped EX_gln__L_e
Flipped EX_so4_e
Flipped EX_thm_e
Flipped EX_thr__L_e
Flipped F6Ptex_1
Flipped ETOHt2r
Flipped G3PD2
Flipped FBA2
Flipped GAMpts
Flipped FF_11
Flipped FORt
Flipped GLUR
Flipped FRUpts
Fl

505

In [51]:
reduced_matrix

<440x505 sparse matrix of type '<class 'numpy.float64'>'
	with 2471 stored elements in List of Lists format>

In [52]:
print(sum(rev_reduced_matrix), 'reversible reactions and', len(rev_reduced_matrix) - sum(rev_reduced_matrix), 'irreversible')

88 reversible reactions and 417 irreversible


In [53]:
original_model = sa.copy()
constrained_model = sa.copy()

In [54]:
import pandas
names = lambda l: list(map(lambda x: x.id, l))

In [55]:
df = pandas.DataFrame.sparse.from_spmatrix(reduced_matrix, index=names(model.metabolites), columns=names(model.reactions))
df

,10M3HDHL,ACALD,ACALDt,3OAS140,3OAS60,ACCOAC,5DOAN,ACGAMT,ACGApts,ACGAptsw,...,EX_hom__L_e,RIBt,EX_hxan_e,TALA,THIORDXi,THMPP,GLUt3,ASPte,ADEt2r,LTAdtarO
10fthf_c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12dgr160_c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12diisgly_c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12napdol_c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13dpg_c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
xan_c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
xmp_c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
xtsn_c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
xtsn_e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
import io
with io.StringIO() as output:
    for column in df.columns:
        dfc = df[column]
        ftime=True
        print(column, ': ', end='', file=output, flush=True)
        for k, v in dfc[dfc < 0].items():
            if ftime:
                ftime=False
            else:
                print('+', end=' ', file=output, flush=True)
            print(-v, k, end=' ', file=output, flush=True)
        print('=' if model.reactions.get_by_id(column).reversibility else '=>', end=' ', file=output, flush=True)
        ftime=True
        for k, v in dfc[dfc > 0].items():
            if ftime:
                ftime=False
            else:
                print('+', end=' ', file=output, flush=True)
            print(v, k, end=' ', file=output, flush=True)
        print(file=output, flush=True)
    lines_r = output.getvalue()
print(lines_r)

10M3HDHL : 2.6724637681159513 3hmrsACP_c + 1.0 3mob_c + 19.0811594202875 3mop_c + 5.895652173911733 4mop_c + 2.6724637681159513 dcaACP_c + 14.324637681156876 glyc3p_c + 460.0811594202044 h_c + 152.46956521736283 malACP_c + 25.976811594199233 nad_c + 307.6115942028416 nadph_c => 129.75877101446736 ACP_c + 178.44637681156206 co2_c + 2.6724637681159513 ddcaACP_c + 155.14202898547876 h2o_c + 25.976811594199233 nadh_c + 307.6115942028416 nadp_c + 12.691628985504993 pi_c + 14.324637681156876 ptdoh_MRSA_c 
ACALD : 1.0 accoa_c + 1.0 h_c + 1.0 nadh_c => 1.0 acald_c + 1.0 coa_c + 1.0 nad_c 
ACALDt : 1.0 acald_c => 
3OAS140 : 1.0 ddcaACP_c + 2.0 h_c + 1.0 malACP_c + 1.0 nadph_c => 1.0 3hmrsACP_c + 1.0 ACP_c + 1.0 co2_c + 1.0 nadp_c 
3OAS60 : 1.0 butACP_c + 6.0 h_c + 2.0 malACP_c + 4.0 nadph_c => 2.0 ACP_c + 2.0 co2_c + 2.0 h2o_c + 4.0 nadp_c + 1.0 ocACP_c 
ACCOAC : 1.0 accoa_c + 1.0 atp_c + 1.0 hco3_c => 1.0 adp_c + 1.0 h_c + 1.0 malcoa_c + 1.0 pi_c 
5DOAN : 1.0 dad_5_c + 1.0 h2o_c => 1.0 ade_c 


In [57]:
rxn_in_sub = [numpy.where(subT[:, i])[0] for i in range(subT.shape[1])]

In [58]:
for i, reactions in enumerate(model.reactions):
    print(f'rsub_{i+1}', list(map(lambda x: constrained_model.reactions[x].id, rxn_in_sub[i])))

rsub_1 ['10M3HDHL', '10M3OACPO', '10M3OUO', '10MDOD', '10MTDAO', '10MTU2A', '10MUAM', '11M3ODO', '11MDAMCA', '11MHDOD', '11MTRDOC', '12M3HTDH', '12M3HTHL', '12M3OXOT', '12M3TDA', '12MTACPM', '12MTT2E', '12TTACA', '3OAS120_2', '13M3HTDAHL', '13M3OTDAO', '13MTDAM', '13MTTDAO', '14M3HDEC', '14M3HPAHL', '14M3OHO', '14M3OP', '14MHDAAD', '14MTACPM', '14MTHEAO', '14MTP2EA', '15M3HEXDA', '3OAS160_2', '3OAS200', '3OXDAM', '4ISOHELS', '4M3G3P', '4M3OHAT', '4M3OPAO', '4M3OXHA', '4METPAC', '4MHEAMA', '4MTHACP', '15M3OHAO', '15MHDAAD', '15MTHACP', '16M3HDEC', '16M3HPAHL', '4MTRPE', '5M3HHAL', '5MET3OH', '5MHACC', '5MTH2EO', '6M3HHL', '16M3OHO', '16M3OP', '16MTHEAO', '16MTP2EA', '17M3HDEC', '17M3OHO', '17MTHEAO', '6M3HOAHL', '6M3OHAO', '6M3OXO', '6MHACPA', '6MOAMCA', '6MTHAO', '6MTR2EO', '7M3HOACPL', '7M3ODO', '7MOACA', '7MTROCAC', '2MACPT', '2MPCAC', '8M3HDAL', '8M3HNAHL', '2MPCTF', '8M3OAO', '8M3ONO', '8MDAM', '8METNO', '8MNAMC', '8MT2ACP', '9M3HDL', '9M3OXPA', '9MDACP', '9MDAM', '3HAD120', '3HAD1

In [59]:
dicto_rsub = {}
biomass = None
for i, reactions in enumerate(model.reactions):
    lm = list(map(lambda x: constrained_model.reactions[x].id, rxn_in_sub[i]))
    if 'BIOMASS_iYS_wild_type' in lm:
        biomass = f'rsub_{i+1}'
    for r in lm:
        dicto_rsub[r] = f'rsub_{i+1}'
print(biomass)

rsub_12


In [60]:
lines = ''
nb_ext = 1
for i, column in enumerate(df.columns):
    dfc = df[column]
    ftime=True
    lines += f'rsub_{i+1}' + ' : '
    for k, v in dfc[dfc < 0].items():
        if ftime:
            ftime=False
        else:
            lines += ' + '
        lines += str(-v) + ' ' + k
    if ftime:
        lines += f'ext_{nb_ext}'
        nb_ext += 1
    lines += ' = ' if model.reactions.get_by_id(column).reversibility else ' => '
    ftime=True
    for k, v in dfc[dfc > 0].items():
        if ftime:
            ftime=False
        else:
            lines += ' + '
        lines += str(v) + ' ' + k
    if ftime:
        lines += f'ext_{nb_ext}'
        nb_ext += 1
    lines += '\n'
lines = '-METEXT\n' + ' '.join(['ext_' + str(i+1) for i in range(nb_ext-1)]) + '\n' + '\n-CAT\n' + lines
print(lines)

-METEXT
ext_1 ext_2 ext_3 ext_4 ext_5 ext_6 ext_7 ext_8 ext_9 ext_10 ext_11 ext_12 ext_13 ext_14 ext_15 ext_16 ext_17 ext_18 ext_19 ext_20 ext_21 ext_22 ext_23 ext_24 ext_25 ext_26 ext_27 ext_28 ext_29 ext_30 ext_31 ext_32 ext_33 ext_34 ext_35 ext_36 ext_37 ext_38 ext_39 ext_40 ext_41 ext_42 ext_43 ext_44 ext_45 ext_46 ext_47 ext_48 ext_49 ext_50

-CAT
rsub_1 : 2.6724637681159513 3hmrsACP_c + 1.0 3mob_c + 19.0811594202875 3mop_c + 5.895652173911733 4mop_c + 2.6724637681159513 dcaACP_c + 14.324637681156876 glyc3p_c + 460.0811594202044 h_c + 152.46956521736283 malACP_c + 25.976811594199233 nad_c + 307.6115942028416 nadph_c => 129.75877101446736 ACP_c + 178.44637681156206 co2_c + 2.6724637681159513 ddcaACP_c + 155.14202898547876 h2o_c + 25.976811594199233 nadh_c + 307.6115942028416 nadp_c + 12.691628985504993 pi_c + 14.324637681156876 ptdoh_MRSA_c
rsub_2 : 1.0 accoa_c + 1.0 h_c + 1.0 nadh_c => 1.0 acald_c + 1.0 coa_c + 1.0 nad_c
rsub_3 : 1.0 acald_c => ext_1
rsub_4 : 1.0 ddcaACP_c + 2.0 h

In [61]:
path = f'{opath}/sa_pa/'

In [62]:
txtfile = path + 'sa_reduced.txt'

In [63]:
with open(txtfile, 'w') as f:
    f.writelines(lines)

In [64]:
ri = lambda i: original_model.reactions[i].id

In [65]:
subname = lambda i: 'rsub_{}'.format(i+1)
subnamerev = lambda i: 'rsub_{}_rev'.format(i+1)
aq = lambda x: '"{}"'.format(x)
aqrev = lambda x: '"{}_rev"'.format(x)
rev = lambda x: '{}_rev'.format(x)  
lines = 'subset(' + ';'.join([aq(subname(i)) for i in range(len(df.columns))]) + ').\n'
lines += 'subset(' + ';'.join([aq(subnamerev(i)) for i in range(len(df.columns))]) + ').\n'
dicto = {}

for r, ls in enumerate(rxn_in_sub):
    dicto[subname(r)] = {'reacs': [ri(rk) if subT[rk, r] >= 0 else rev(ri(rk)) for rk in ls],
         'coeffs': [abs(subT[rk, r]) for rk in ls],
         'full': [(ri(rk), subT[rk, r]) for rk in ls]}
    if model.reactions.get_by_any(r)[0].reversibility:
        dicto[subnamerev(r)] = {'reacs': [ri(rk) if subT[rk, r] < 0 else rev(ri(rk)) for rk in ls],
             'coeffs': [abs(subT[rk, r]) for rk in ls],
             'full': [(ri(rk), -subT[rk, r]) for rk in ls]}
    for rk in ls:
        fk = subT[rk, r]
        lines += 'coefficient(' + aq(subname(r)) + ',' + ((aq(ri(rk))) if fk >= 0 else aqrev(ri(rk))) + ',' + aq(abs(fk)) + ').\n'
        lines += 'coefficient(' + aq(subnamerev(r)) + ',' + ((aqrev(ri(rk))) if fk >= 0 else aq(ri(rk))) + ',' + aq(abs(fk)) + ').\n'

In [66]:
lp4file = path + 'sa_reactionSubsets.lp4'
subfile = path + 'sa_reactionSubsets.txt'

In [67]:
with open(lp4file, 'w') as f:
    f.writelines(lines)

In [68]:
with open(subfile, 'w') as f:
    f.write(str(dicto))

In [69]:
mcssubname = lambda i: '"mcs_rsub_{}"'.format(i+1)
mcssubnamerev = lambda i: '"mcs_rsub_{}_rev"'.format(i+1)
aq = lambda x: '"{}"'.format(x)
aqrev = lambda x: '"{}_rev"'.format(x)    
lines = 'subset(' + ';'.join([mcssubname(i) for i in range(len(df.columns))]) + ').\n'
lines += 'subset(' + ';'.join([mcssubnamerev(i) for i in range(len(df.columns))]) + ').\n'

for r, ls in enumerate(rxn_in_sub):
    for rk in ls:
        fk = subT[rk, r]
        lines += 'coefficient(' + mcssubname(r) + ',' + ((aq(ri(rk))) if fk >= 0 else aqrev(ri(rk))) + ',' + aq(abs(fk)) + ').\n'
        lines += 'coefficient(' + mcssubnamerev(r) + ',' + ((aqrev(ri(rk))) if fk >= 0 else aq(ri(rk))) + ',' + aq(abs(fk)) + ').\n'

In [70]:
lp4file = path + 'sa_mcsReactionSubsets.lp4'

In [71]:
with open(lp4file, 'w') as f:
    f.writelines(lines)

In [72]:
len(original_model.reactions)

1454

In [73]:
lines = '&dom{LB..UB} = flux(R) :- reaction(R); bounds(R, LB, UB).\n'
for i, r in enumerate(model.reactions):
    if r.upper_bound < 0:
        lines += 'bounds(' + aq(subnamerev(i)) + ',' + aq(-r.lower_bound) + ',' + aq(-r.upper_bound) + ').\n'
    elif r.lower_bound < 0:
        lines += 'bounds(' + aq(subname(i)) + ',' + aq(0) + ',' + aq(r.upper_bound) + ').\n'
        lines += 'bounds(' + aq(subnamerev(i)) + ',' + aq(0) + ',' + aq(-r.lower_bound) + ').\n'
    else:
        lines += 'bounds(' + aq(subname(i)) + ',' + aq(r.lower_bound) + ',' + aq(r.upper_bound) + ').\n'

In [74]:
bndfile = path + 'sa_reduced_bounds.lp4'

In [75]:
with open(bndfile, 'w') as f:
    f.writelines(lines)

In [76]:
txtfile=f'{path}sa_reduced.txt'
pklfile=f'{path}sa_reduced.pkl'
aspfile=f'{path}sa_reduced.lp4'
sbmlfile=f'{path}sa_reduced.xml'
mcs_pklfile=f'{path}sa_reduced_mcs.pkl'
mcs_aspfile=f'{path}sa_reduced_mcs.lp4'
mcs_sbmlfile=f'{path}sa_reduced_mcs.xml'

In [77]:
mparser.convert(mparser.conversion.InputFormatType.TXT, txtfile , mparser.conversion.OutputFormatType.PKL, pklfile, False, [], False)
mparser.convert(mparser.conversion.InputFormatType.TXT, txtfile , mparser.conversion.OutputFormatType.ASP, aspfile, False, [], False)
mparser.convert(mparser.conversion.InputFormatType.TXT, txtfile , mparser.conversion.OutputFormatType.SBML, sbmlfile, False, [], False)

279 internal metabolites
50 external metabolites
505 total reactions
50 exchange reactions
88 reversible reactions
279 internal metabolites
50 external metabolites
505 total reactions
50 exchange reactions
88 reversible reactions
279 internal metabolites
50 external metabolites
505 total reactions
50 exchange reactions
88 reversible reactions


In [78]:
mparser.convert(mparser.conversion.InputFormatType.TXT, txtfile , mparser.conversion.OutputFormatType.PKL, mcs_pklfile, to_dual_mcs=True, target_reactions=[biomass], ballerstein=False)
mparser.convert(mparser.conversion.InputFormatType.TXT, txtfile , mparser.conversion.OutputFormatType.ASP, mcs_aspfile, to_dual_mcs=True, target_reactions=[biomass], ballerstein=False)
mparser.convert(mparser.conversion.InputFormatType.TXT, txtfile , mparser.conversion.OutputFormatType.SBML, mcs_sbmlfile, to_dual_mcs=True, target_reactions=[biomass], ballerstein=False)

279 internal metabolites
50 external metabolites
505 total reactions
50 exchange reactions
88 reversible reactions


/home/maxime/Bureau/Thesis/wksp/github/paSAmcs/mparser/mparser/meta_network.py:202: UserWarning: No external metabolite definition
  self.structures_check()
/home/maxime/Bureau/Thesis/wksp/github/paSAmcs/mparser/mparser/meta_network.py:43: UserWarning: Found reactions with missing overall metabolite production or consumption
  warnings.warn('Found reactions with missing overall metabolite production or consumption')


-- Dual network --
505 internal metabolites
0 external metabolites
785 total reactions
50 exchange reactions
367 reversible reactions
279 internal metabolites
50 external metabolites
505 total reactions
50 exchange reactions
88 reversible reactions
-- Dual network --
505 internal metabolites
0 external metabolites
785 total reactions
50 exchange reactions
367 reversible reactions
279 internal metabolites
50 external metabolites
505 total reactions
50 exchange reactions
88 reversible reactions
-- Dual network --
505 internal metabolites
0 external metabolites
785 total reactions
50 exchange reactions
367 reversible reactions


In [79]:
efmcfile=f'{path}sa_reduced.efmc'

In [80]:
from prepare_efm_checker import compute_efm_checker

In [81]:
compute_efm_checker(pklfile, efmcfile, None, True)

/home/maxime/Bureau/Thesis/wksp/github/paSAmcs/mparser/aspsmatrix.py:219: UserWarning: large model; skipping nullspace computation
  warnings.warn("large model; skipping nullspace computation")
